# setup

In [1]:
import requests
import gzip
import shutil
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.pyplot as plt

In [5]:
# todo needs a fix so it can be usefull

base_url = "https://datasets.imdbws.com/"
title_basics_file = "title.basics.tsv"
title_basics_zip = "title.basics.tsv.gz"

BASE_URL = "https://datasets.imdbws.com/"
FILES = {
    "tit_bas_f": "title.basics.tsv",
    "tit_bas_z": "title.basics.tsv.gz",
    "tit_rate_f": "title.ratings.tsv",
    "tit_rate_z": "title.ratings.tsv.gz",
    "tit_crew_f": "title.crew.tsv",
    "tit_crew_z": "title.crew.tsv.gz",
    "name_bas_f": "name.basics.tsv",
    "name_bas_z": "name.basics.tsv.gz",
    "cast_crew_f": "title.principals.tsv",
    "cast_crew_z": "title.principals.tsv.gz"
    }
    

## Downloading files

In [6]:
# Change to True is you want to update the imdb files
DOWNLOAD = False

In [7]:
# removes old files if already excist
if DOWNLOAD:
    files = ["title.basics.tsv", "title.ratings.tsv", "title.crew.tsv", "name.basics.tsv", "title.principals.tsv"]

    if not os.path.exists("data"):
        os.makedirs("data")
    
    for file in files:
        file_name = os.path.join("data/",file)
        file_zip = file_name+".gz"
        file_url = BASE_URL+file+".gz"
        
        # remove old files
        if os.path.exists(file_name):
            os.remove(file_name)
        if os.path.exists(file_zip):
            os.remove(file_zip)
        
        # download files
        response = requests.get(file_url)
        open(file_zip , "wb").write(response.content)

        # unzip files
        with gzip.open(file_zip, 'rb') as f_in:
            with open(file_name, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        
        # remove zips
        os.remove(file_zip)

# Creating a raw dataset for mining

## Adding new movies to raw_watched

In [5]:
# getting the new seen movies
seen_path = "data/add_movies_seen.txt"
seen_raw_f = open(seen_path,'r')
seen_raw = seen_raw_f.readlines()

# transforming the new seen movie data
for linei in range(len(seen_raw)):
    seen_raw[linei] = seen_raw[linei].strip().split(" ")
    if len(seen_raw[linei]) == 1:
        temp = seen_raw[linei][0].split("/")
    elif len(seen_raw[linei]) == 2:
        temp = seen_raw[linei][0].split("/")
    tofind = re.compile("^tt\d+\d$")
    ttcode = ""
    for x in temp:
        y = tofind.findall(x)
        if len(list(y)) != 0:
            ttcode = list(y)[0]
    if len(seen_raw[linei]) == 1:
        seen_raw[linei] = [ttcode, None]
    elif len(seen_raw[linei]) == 2:
        seen_raw[linei] = [ttcode, int(seen_raw[linei][1])]
seen_raw


[['tt0054135', None],
 ['tt0903624', None],
 ['tt1170358', None],
 ['tt2310332', None],
 ['tt0120363', None],
 ['tt1630029', None],
 ['tt0050083', 4],
 ['tt8579674', 4]]

In [6]:
# getting the new unseen movies
unseen_path = "data/add_movies_unseen.txt"
unseen_raw_f = open(unseen_path,'r')
unseen_raw = unseen_raw_f.readlines()

# transforming the new unseen movie data
for linei in range(len(unseen_raw)):
    unseen_raw[linei] = unseen_raw[linei].strip()
    temp = unseen_raw[linei].split("/")
    
    tofind = re.compile("^tt\d+\d$")
    ttcode = ""
    for x in temp:
        y = tofind.findall(x)
        if len(list(y)) != 0:
            ttcode = list(y)[0]
    unseen_raw[linei] = ttcode
unseen_raw

['tt1462764',
 'tt6791350',
 'tt15239678',
 'tt9362722',
 'tt11210390',
 'tt15843316',
 'tt10366206',
 'tt2584384']

Adding the 2d list to the raw_watched.xlsx (if not already added)

In [7]:
# tt1630029 already in it
# tt8579674 already in it has 3 will not be 4

ids_and_status = 'data/raw_watched.xlsx'
movie_list_raw = pd.read_excel(ids_and_status)
movie_list_raw

,tconst,watched,netflix,prime,enjoyment,priority
0,tt0015324,0,NaN,NaN,NaN,NaN
1,tt0017136,0,0.0,0.0,NaN,NaN
2,tt0022100,0,0.0,0.0,NaN,NaN
3,tt0025316,0,NaN,NaN,NaN,NaN
4,tt0031381,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
595,tt9691136,0,0.0,0.0,NaN,NaN
596,tt9731598,0,0.0,0.0,NaN,NaN
597,tt9764362,1,0.0,0.0,4.0,NaN
598,tt9783600,1,NaN,NaN,4.0,NaN


In [8]:
# occuring viewed score given                   "tt8579674" 4 > "tt8579674" 1 3         oke
# occuring viewed score not given               "tt0322259" nan > "tt0322259" 1 nan     oke 
# occuring not viewed score given               "tt1293847" 4 > "tt1293847" 1 4         oke
# occuring not viewed score not given           "tt0055630" nan > "tt0055630" 1 nan     oke 
# not occuring viewed score given               "tt66" 4 > "tt66"  1 4                  oke
# not occuring viewed score not given           "tt66"  > "tt66" 1                      oke
# not occuring not viewed score given           "tt66" 4 > "tt66" 1 4                   -
# not occuring not viewed score not given       "tt66"  > "tt66" 1                      -

for seen in seen_raw:
    movieid = seen[0]
    score = seen[1]
    if movieid in movie_list_raw["tconst"].values:
        # if watched movie already in list
        found_index = movie_list_raw.loc[movie_list_raw.loc[:,"tconst"]==movieid].index.tolist()[0]
        movie_list_raw.at[found_index,"enjoyment"]
        enjoyment = movie_list_raw.at[found_index,"enjoyment"]
        watched = int(movie_list_raw.at[found_index,"watched"])
        if watched==1 and pd.isnull(enjoyment):
            # update the score only if null
            movie_list_raw.at[found_index,"enjoyment"] = score
        elif watched==0:
            # updated watched and add score (which can be nan)
            movie_list_raw.at[found_index,"enjoyment"] = score
            movie_list_raw.at[found_index,"watched"] = 1
    else:
        # if watched movie not in list
        to_add = pd.Series({
            'tconst':movieid, 'watched':1, 'netflix':np.nan,
            'prime':np.nan, "enjoyment":score , "priority": np.nan})

        movie_list_raw = pd.concat([movie_list_raw, to_add.to_frame().T], ignore_index=True)

    # movie_list_raw.loc[movie_list_raw.loc[:,"tconst"]==movieid]    

In [9]:
for movieid in unseen_raw:
    if not movieid in movie_list_raw["tconst"].values:
        # if watched movie not already in list
        to_add = pd.Series({
            'tconst':movieid, 'watched':0, 'netflix':np.nan,
            'prime':np.nan, "enjoyment":np.nan , "priority": np.nan})

        movie_list_raw = pd.concat([movie_list_raw, to_add.to_frame().T], ignore_index=True)

In [10]:
movie_list_raw.sort_values(["tconst"]).to_excel("data/raw_watched.xlsx", index=False)

## Clean watched data

enjoyment score: 0=no; 1=mweh; 2=fun; 3=good; 4=great

In [11]:
ids_and_status = 'data/raw_watched.xlsx'
watched = pd.read_excel(ids_and_status)
watched["watched"] = watched["watched"].replace(to_replace=0.0, value=False).replace(to_replace=1.0, value=True)
watched["prime"] = watched["prime"].replace(to_replace=0.0, value=False).replace(to_replace=1.0, value=True)
watched["netflix"] = watched["netflix"].replace(to_replace=0.0, value=False).replace(to_replace=1.0, value=True)
watched["enjoyment"] = watched["enjoyment"]
watched["tconst"] = watched["tconst"].str.strip()
watched

,tconst,watched,netflix,prime,enjoyment,priority
0,tt0015324,False,NaN,NaN,NaN,NaN
1,tt0017136,False,False,False,NaN,NaN
2,tt0022100,False,False,False,NaN,NaN
3,tt0025316,False,NaN,NaN,NaN,NaN
4,tt0031381,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
595,tt9691136,False,False,False,NaN,NaN
596,tt9731598,False,False,False,NaN,NaN
597,tt9764362,True,False,False,4.0,NaN
598,tt9783600,True,NaN,NaN,4.0,NaN


## Add imdb data

### Add title data

In [12]:
title_basics_file = os.path.join("data/",FILES["tit_bas_f"])
title_basics = pd.read_csv(title_basics_file, sep="\t")
title_basics = title_basics.replace(to_replace = "\\N", value = np.nan)
title_watched = pd.merge(watched, title_basics, on="tconst", how="left") # new merge, keeps wrong stuff
title_watched

/tmp/ipykernel_13248/2144297984.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv(title_basics_file, sep="\t")


,tconst,watched,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0015324,False,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,0,1924,NaN,45,"Action,Comedy,Romance"
1,tt0017136,False,False,False,NaN,NaN,movie,Metropolis,Metropolis,0,1927,NaN,153,"Drama,Sci-Fi"
2,tt0022100,False,False,False,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,0,1931,NaN,117,"Crime,Mystery,Thriller"
3,tt0025316,False,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,0,1934,NaN,105,"Comedy,Romance"
4,tt0031381,False,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,0,1939,NaN,238,"Drama,Romance,War"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,tt9691136,False,False,False,NaN,NaN,movie,Shadow in the Cloud,Shadow in the Cloud,0,2020,NaN,83,"Action,Horror,War"
596,tt9731598,False,False,False,NaN,NaN,movie,Bros,Bros,0,2022,NaN,115,"Comedy,Romance"
597,tt9764362,True,False,False,4.0,NaN,movie,The Menu,The Menu,0,2022,NaN,107,"Comedy,Horror,Thriller"
598,tt9783600,True,NaN,NaN,4.0,NaN,movie,Spiderhead,Spiderhead,0,2022,NaN,106,"Action,Crime,Drama"


### Add scores


In [13]:
title_rate_file = os.path.join("data/",FILES["tit_rate_f"])
title_rate = pd.read_csv(title_rate_file, sep="\t")
title_rate = title_rate.replace(to_replace = "\\N", value = np.nan)
watched_film_fin = pd.merge(title_watched,title_rate, on="tconst", how="left")
watched_film_fin

,tconst,watched,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0015324,False,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,0,1924,NaN,45,"Action,Comedy,Romance",8.2,50693.0
1,tt0017136,False,False,False,NaN,NaN,movie,Metropolis,Metropolis,0,1927,NaN,153,"Drama,Sci-Fi",8.3,175193.0
2,tt0022100,False,False,False,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,0,1931,NaN,117,"Crime,Mystery,Thriller",8.3,158817.0
3,tt0025316,False,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,0,1934,NaN,105,"Comedy,Romance",8.1,104812.0
4,tt0031381,False,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,0,1939,NaN,238,"Drama,Romance,War",8.2,316504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,tt9691136,False,False,False,NaN,NaN,movie,Shadow in the Cloud,Shadow in the Cloud,0,2020,NaN,83,"Action,Horror,War",5.0,27461.0
596,tt9731598,False,False,False,NaN,NaN,movie,Bros,Bros,0,2022,NaN,115,"Comedy,Romance",6.4,21948.0
597,tt9764362,True,False,False,4.0,NaN,movie,The Menu,The Menu,0,2022,NaN,107,"Comedy,Horror,Thriller",7.5,37049.0
598,tt9783600,True,NaN,NaN,4.0,NaN,movie,Spiderhead,Spiderhead,0,2022,NaN,106,"Action,Crime,Drama",5.4,56215.0


### Add directors

In [14]:
crew_file = os.path.join("data/",FILES["tit_crew_f"])
crew = pd.read_csv(crew_file, sep="\t")
crew = crew.replace(to_replace = "\\N", value = np.nan)
watched_film_fin2 = pd.merge(watched_film_fin,crew, on="tconst", how="left")
watched_film_fin2

,tconst,watched,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
0,tt0015324,False,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,0,1924,NaN,45,"Action,Comedy,Romance",8.2,50693.0,nm0000036,"nm0369841,nm0593477,nm0115669"
1,tt0017136,False,False,False,NaN,NaN,movie,Metropolis,Metropolis,0,1927,NaN,153,"Drama,Sci-Fi",8.3,175193.0,nm0000485,"nm0902376,nm0000485"
2,tt0022100,False,False,False,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,0,1931,NaN,117,"Crime,Mystery,Thriller",8.3,158817.0,nm0000485,"nm0902376,nm0000485,nm0414825"
3,tt0025316,False,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,0,1934,NaN,105,"Comedy,Romance",8.1,104812.0,nm0001008,"nm0728307,nm0011343"
4,tt0031381,False,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,0,1939,NaN,238,"Drama,Romance,War",8.2,316504.0,"nm0281808,nm0002030,nm0939992","nm0593565,nm0397608,nm0308177,nm0372942,nm0842..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,tt9691136,False,False,False,NaN,NaN,movie,Shadow in the Cloud,Shadow in the Cloud,0,2020,NaN,83,"Action,Horror,War",5.0,27461.0,nm1755396,"nm0484840,nm1755396"
596,tt9731598,False,False,False,NaN,NaN,movie,Bros,Bros,0,2022,NaN,115,"Comedy,Romance",6.4,21948.0,nm0831557,"nm2813994,nm0831557"
597,tt9764362,True,False,False,4.0,NaN,movie,The Menu,The Menu,0,2022,NaN,107,"Comedy,Horror,Thriller",7.5,37049.0,nm0617042,"nm2219721,nm4301557"
598,tt9783600,True,NaN,NaN,4.0,NaN,movie,Spiderhead,Spiderhead,0,2022,NaN,106,"Action,Crime,Drama",5.4,56215.0,nm2676052,"nm2808422,nm1014201,nm1116660"


### Un-list the genres

In [15]:
def clean_genre_list(genre):
    if isinstance(genre, str):
        return genre.split(",")
    else:
        return np.nan

watched_film_fin2["genres"] = watched_film_fin2["genres"].apply(clean_genre_list)

In [16]:
usable = watched_film_fin2[~watched_film_fin2["titleType"].isna()]
unusable = watched_film_fin2[watched_film_fin2["titleType"].isna()]

In [17]:
# https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
def boolean_df(item_lists, unique_items):# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

def genres_to_list_1D(series):
        return pd.Series([x for _list in series for x in _list])

In [18]:
genres = genres_to_list_1D(usable["genres"]).value_counts()
genres_bool = boolean_df(usable["genres"], genres.keys().sort_values())
keys = genres.keys().sort_values()
keys_new = ["genre_"+key for key in keys]
keys_change_dict = {}
for x in range(len(keys)):
    keys_change_dict[keys[x]] = keys_new[x]
genres_bool = genres_bool.rename(columns=keys_change_dict)
usable = usable.join(genres_bool)
usable.drop('genres', axis=1, inplace=True)

In [19]:
unusable

,tconst,watched,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers


In [20]:
usable

,tconst,watched,netflix,prime,enjoyment,priority,titleType,primaryTitle,originalTitle,isAdult,...,genre_Music,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Short,genre_Sport,genre_Thriller,genre_War,genre_Western
0,tt0015324,False,NaN,NaN,NaN,NaN,movie,Sherlock Jr.,Sherlock Jr.,0,...,False,False,False,True,False,False,False,False,False,False
1,tt0017136,False,False,False,NaN,NaN,movie,Metropolis,Metropolis,0,...,False,False,False,False,True,False,False,False,False,False
2,tt0022100,False,False,False,NaN,NaN,movie,M,M - Eine Stadt sucht einen Mörder,0,...,False,False,True,False,False,False,False,True,False,False
3,tt0025316,False,NaN,NaN,NaN,NaN,movie,It Happened One Night,It Happened One Night,0,...,False,False,False,True,False,False,False,False,False,False
4,tt0031381,False,NaN,NaN,NaN,NaN,movie,Gone with the Wind,Gone with the Wind,0,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,tt9691136,False,False,False,NaN,NaN,movie,Shadow in the Cloud,Shadow in the Cloud,0,...,False,False,False,False,False,False,False,False,True,False
596,tt9731598,False,False,False,NaN,NaN,movie,Bros,Bros,0,...,False,False,False,True,False,False,False,False,False,False
597,tt9764362,True,False,False,4.0,NaN,movie,The Menu,The Menu,0,...,False,False,False,False,False,False,False,True,False,False
598,tt9783600,True,NaN,NaN,4.0,NaN,movie,Spiderhead,Spiderhead,0,...,False,False,False,False,False,False,False,False,False,False


### split directors and writers (could also take only the first two of both, if handy?)

In [21]:
# usable["directors"] = usable["directors"].apply(lambda x: x.split(',')[0:2])
usable["directors"] = usable["directors"].apply(lambda x: x.split(','))
# unusable2 = usable[usable["writers"].map(type)==float]
# usable.loc[usable["writers"].map(type)!=float, "writers"] = usable.loc[usable["writers"].map(type)!=float, "writers"].apply(lambda x: x.split(',')[0:2])
usable.loc[usable["writers"].map(type)!=float, "writers"] = usable.loc[usable["writers"].map(type)!=float, "writers"].apply(lambda x: x.split(','))

### explode two times on director and writer.
At max this should give 4 duplicates for a movie (we only took the first two directors and writers)

In [22]:
usable = usable.explode("writers").explode("directors")
usable[["tconst", "directors","writers"]]

,tconst,directors,writers
0,tt0015324,nm0000036,nm0369841
0,tt0015324,nm0000036,nm0593477
0,tt0015324,nm0000036,nm0115669
1,tt0017136,nm0000485,nm0902376
1,tt0017136,nm0000485,nm0000485
...,...,...,...
598,tt9783600,nm2676052,nm2808422
598,tt9783600,nm2676052,nm1014201
598,tt9783600,nm2676052,nm1116660
599,tt9806192,nm3021346,nm3021346


## The following dataset can be used for mining (if accounted for multiple directors and writers)

In [23]:
usable.replace(True, 1).replace(False,0).to_excel("data/films_for_mining.xlsx",index=True)

# Make version of dataset that is human readable

## Load the names

In [24]:
names_file = os.path.join("data/",FILES["name_bas_f"])
names_basics = pd.read_csv(names_file, sep="\t")
names_basics = names_basics.replace(to_replace = "\\N", value = np.nan)
col_delete = ["knownForTitles", "primaryProfession"]
names_basics = names_basics.drop(col_delete, axis=1)
names_basics

,nconst,primaryName,birthYear,deathYear
0,nm0000001,Fred Astaire,1899,1987
1,nm0000002,Lauren Bacall,1924,2014
2,nm0000003,Brigitte Bardot,1934,NaN
3,nm0000004,John Belushi,1949,1982
4,nm0000005,Ingmar Bergman,1918,2007
...,...,...,...,...
12146505,nm9993714,Romeo del Rosario,NaN,NaN
12146506,nm9993716,Essias Loberg,NaN,NaN
12146507,nm9993717,Harikrishnan Rajan,NaN,NaN
12146508,nm9993718,Aayush Nair,NaN,NaN


## Add directors and writers

In [25]:
usable = pd.merge(usable,names_basics, left_on="directors", right_on="nconst", how="left")
usable = pd.merge(usable,names_basics, left_on="writers", right_on="nconst", how="left")

## Clean output

Hier ben ik gebeleven. Moet nog een keer kijken naar welke kolommen ik ga weg gooien of hernoemen

In [26]:
col_delete = ['isAdult', 'endYear', 'directors', 'writers', 'nconst_x', 'nconst_y']
usable = usable.drop(col_delete, axis=1)

## Reverse the explosion

Get columns

In [27]:
usable_rename = {   "primaryName_x": "director", "birthYear_x": "director_birthYear", "deathYear_x": "director_deathYear",
                    "primaryName_y": "writer", "birthYear_y": "writer_birthYear", "deathYear_y": "writer_deathYear"}
usable = usable.rename(columns=usable_rename)
all_column_titles = list(usable.columns)
column_titles_collapse = ['tconst', 'director', 'director_birthYear','director_deathYear', 'writer', 'writer_birthYear', 'writer_deathYear']
colums_titles_unique = [e for e in all_column_titles if e not in column_titles_collapse]
colums_titles_unique.insert(0,'tconst')

Get unique rows and duplicate rows

In [28]:
unique_cols = usable[colums_titles_unique].drop_duplicates()
dup_cols = usable[column_titles_collapse]

In [29]:
unique_cols = usable[colums_titles_unique].drop_duplicates()
dup_cols = usable[column_titles_collapse]
dup_cols.loc[:,"directors"] = dup_cols.loc[:,"director"].astype(str)+" "+dup_cols.loc[:,"director_birthYear"].fillna('').astype(str)+"-"+dup_cols.loc[:,"director_deathYear"].fillna('').astype(str)
dup_cols.loc[:,"writers"] = dup_cols.loc[:,"writer"].astype(str)+" "+dup_cols.loc[:,"writer_birthYear"].fillna('').astype(str)+"-"+dup_cols.loc[:,"writer_deathYear"].fillna('').astype(str)
delete_cols = ['director', 'director_birthYear','director_deathYear', 'writer', 'writer_birthYear', 'writer_deathYear']
dup_cols = dup_cols.drop(delete_cols, axis=1)

/tmp/ipykernel_13248/464737682.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup_cols.loc[:,"directors"] = dup_cols.loc[:,"director"].astype(str)+" "+dup_cols.loc[:,"director_birthYear"].fillna('').astype(str)+"-"+dup_cols.loc[:,"director_deathYear"].fillna('').astype(str)
/tmp/ipykernel_13248/464737682.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dup_cols.loc[:,"writers"] = dup_cols.loc[:,"writer"].astype(str)+" "+dup_cols.loc[:,"writer_birthYear"].fillna('').astype(str)+"-"+dup_cols.loc[:,"wr

group the data back again

In [30]:
# https://stackoverflow.com/questions/64235312/how-to-implodereverse-of-pandas-explode-based-on-a-column
dup_cols = dup_cols.groupby('tconst').agg({'directors': lambda x: x.tolist(),'writers': lambda x: x.tolist()})
dup_cols["directors"] = dup_cols["directors"].map(np.unique)
dup_cols["writers"] = dup_cols["writers"].map(np.unique)

add the director and writer data back to the set

In [31]:
fin = pd.merge(unique_cols, dup_cols, on="tconst", how="left")

Add unusable data back and delete the columns that got back, because of the merge.

In [32]:
fin = pd.concat([fin, unusable])
delete_cols = ['isAdult', 'endYear', 'genres']
fin = fin.drop(delete_cols, axis=1)

replace True and False with a's and 0's

In [33]:
fin = fin.replace(True, 1).replace(False,0)

/home/user/ManualInstalls/anaconda3/envs/ML/lib/python3.10/site-packages/pandas/core/missing.py:95: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  new_mask = arr == x


In [34]:
fin.sort_values(by=['watched', 'averageRating'], ascending=False).to_excel("data/films_for_reading.xlsx",index=False)

API voor imdb code naar netflix en prime
https://rapidapi.com/utelly/api/Utelly

https://www.finder.com/netflix-movies

https://www.filmvandaag.nl/video-on-demand/netflix/volledig-aanbod?categorie=films